<small><i>This notebook was put together by Fabian Pedregosa based on work by [Jake Vanderplas](http://www.vanderplas.com).</i></small>

# Supervised learning I

Here we'll dive into the basic principles of machine learning, and how to
utilize them via the Scikit-Learn API.

After briefly introducing scikit-learn's *Estimator* object, we'll cover **supervised learning**, including *classification* and *regression* problems, and **unsupervised learning**, including *dimensinoality reduction* and *clustering* problems.

In [ ]:
# some imports

# plotting, set up plotly offline
from plotly.offline import iplot, init_notebook_mode
init_notebook_mode()
import plotly.graph_objs as go

# numpy
import numpy as np

## The Scikit-learn Estimator Object

Every algorithm is exposed in scikit-learn via an ''Estimator'' object. For instance a linear regression is implemented as so:

In [ ]:
from sklearn.linear_model import LinearRegression

**Estimator parameters**: All the parameters of an estimator can be set when it is instantiated, and have suitable default values:

In [ ]:
lin_reg = LinearRegression(normalize=True)
print(lin_reg)

**Estimated Model parameters**: When data is *fit* with an estimator, parameters are estimated from the data at hand. But first, lets create some data

In [ ]:
x = np.arange(10)
y = 2 * x + 1

In [ ]:
print(x)
print(y)

In [ ]:
# Plot the data and embed in the notebook!
iplot([go.Scatter(x=x, y=y)])

In [ ]:
# The input data for sklearn is 2D: (samples == 10 x features == 1)
X = x[:, np.newaxis]
print(X)
print(y)

In [ ]:
# fit the model on our data
lin_reg.fit(X, y)


Now the model is estimated. Hurray!.

All the estimated parameters are attributes of the estimator object ending by an underscore.

In [ ]:
# underscore at the end indicates a fit parameter
print(lin_reg.coef_)
print(lin_reg.intercept_)

The great thing now is that we can use this model to "predict":

In [ ]:
lin_reg.predict(X)

We will see how to use this in the next section.

### Exercise

Repeat the above but using a ```Ridge``` model intead of ```LinearRegression```. Does the prediction change?

## Supervised Learning: Regression and Classification

What we have just seen is a simple instance of supervised learning. We will now see some more challenging scenarios. But first, what is supervised learning anyway?

In **Supervised Learning**, we have a dataset consisting of both features and labels.
The task is to construct an estimator which is able to predict the label of an object
given the set of features. Some examples are:

- given a multicolor image of an object through a telescope, determine
  whether that object is a star, a quasar, or a galaxy.
- given a photograph of a person, identify the person in the photo.
- given a list of movies a person has watched and their personal rating
  of the movie, recommend a list of movies they would like
  (So-called *recommender systems*: a famous example is the [Netflix Prize](http://en.wikipedia.org/wiki/Netflix_prize)).

What these tasks have in common is that there is one or more unknown
quantities associated with the object which needs to be determined from other
observed quantities.

Supervised learning is further broken down into two categories, **classification** and **regression**.
In classification, the label is discrete, while in regression, the label is continuous. For example,
in astronomy, the task of determining whether an object is a star, a galaxy, or a quasar is a
classification problem: the label is from three distinct categories. On the other hand, we might
wish to estimate the age of an object based on such observations: this would be a regression problem,
because the label (age) is a continuous quantity.

---

### Regression Example

One of the simplest regression problems is fitting a line to data, which we saw above.
Scikit-learn also contains more sophisticated regression algorithms

In [ ]:
# Create some simple data
import numpy as np
np.random.seed(0)
X = np.random.random(size=(20, 1))

# y = linear model + noise
y = 3 * X.ravel() + 2 + np.random.randn(20)

iplot([go.Scatter(x=X.ravel(), y=y, mode='markers')])

As above, we can plot a line of best fit:

In [ ]:
lin_reg = LinearRegression()
lin_reg.fit(X, y)

X_all = np.linspace(0, 1, 100)[:, np.newaxis]
y_predict = lin_reg.predict(X_all)

iplot([
        go.Scatter(x=X.ravel(), y=y, mode='markers', name='original data'),
        go.Scatter(x=X_all.ravel(), y=y_predict, name='estimated model')
    ])

Scikit-learn also has some more sophisticated models, which can respond to finer features in the data:

In [ ]:
# Fit a Random Forest
from sklearn.ensemble import RandomForestRegressor
ran_for = RandomForestRegressor()
ran_for.fit(X, y)

# Plot the data and the model prediction
X_all = np.linspace(0, 1, 100)[:, np.newaxis]
y_predict = ran_for.predict(X_all)

iplot([
        go.Scatter(x=X.ravel(), y=y, mode='markers', name='original data'),
        go.Scatter(x=X_all.ravel(), y=y_predict, name='estimated model')
    ])

We can compare the performance of the different classifiers using the method .score(X, y). This returns a number between [-1, 1] representing the accuracy of the model (higher is better).

In [ ]:
lin_reg.score(X, y)

In [ ]:
ran_for.score(X, y)

---

#### Exercise

Explore the ``RandomForestRegressor`` object using IPython's help features (i.e. put a question mark after the object).
What arguments are available to ``RandomForestRegressor``?
How does the above plot change if you change these arguments?

These class-level arguments are known as *hyperparameters*, and we will discuss later how you to select hyperparameters in the model validation section.

---

## Classification

Suppose now that instead of predicting a real number you want to predict a categorial variable. For example, yes/no (2 classes), yes/no/maybe (3 classes), etc.

We will consider an example dataset with 3 classes. Its a classical example in which the task is to predict from 3 species of iris (a flower) given a set of measurements of its flower. 

K nearest neighbors (kNN) is one of the simplest learning strategies: given a new, unknown observation, look up in your reference database which ones have the closest features and assign the predominant class.
Let's try it out on our iris classification problem:

In [ ]:
# the dataset
from sklearn import datasets, neighbors
iris = datasets.load_iris()
X = iris.data
y = iris.target

In [ ]:
# create the model
knn = neighbors.KNeighborsClassifier(n_neighbors=5)

# fit the model
knn.fit(X, y)

# What kind of iris has 3cm x 5cm sepal and 4cm x 2cm petal?
# call the "predict" method:
result = knn.predict([[3, 5, 4, 2],])

print(iris.target_names[result])

### Exercise
Use a different estimator on the same problem: sklearn.svm.SVC.

Note that you don't have to know what it is do use it. We're simply trying out the interface here

### Recap: Scikit-learn's estimator interface

Scikit-learn strives to have a uniform interface across all methods,
and we'll see examples of these below. Given a scikit-learn *estimator*
object named `model`, the following methods are available:

- For all **supervised estimators**
  + `model.fit()` : fit training data. For supervised learning applications,
    this accepts two arguments: the data `X` and the labels `y` (e.g. `model.fit(X, y)`).
    For unsupervised learning applications, this accepts only a single argument,
    the data `X` (e.g. `model.fit(X)`).
  + `model.predict()` : given a trained model, predict the label of a new set of data.
    This method accepts one argument, the new data `X_new` (e.g. `model.predict(X_new)`),
    and returns the learned label for each object in the array.
  + `model.score()` : for classification or regression problems, all estimators implement
    a score method.  Scores are between 0 and 1, with a larger score indicating a better fit.

- Some also implement:
  + `model.predict_proba()` : For classification problems, some estimators also provide
    this method, which returns the probability that a new observation has each categorical label.
    In this case, the label with the highest probability is returned by `model.predict()`.


## Model Validation

An important piece of machine learning is **model validation**: that is, determining how well your model will generalize from the training data to future unlabeled data. Let's look at an example using the *nearest neighbor classifier*. This is a very simple classifier: it simply stores all training data, and for any unknown quantity, simply returns the label of the closest training point.

With the iris data, it very easily returns the correct prediction for each of the input points:

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
X, y = iris.data, iris.target
clf = KNeighborsClassifier(n_neighbors=1)
clf.fit(X, y)
print(clf.score(X, y))


For each class, all 50 training samples are correctly identified. But this **does not mean that our model is perfect!** In particular, such a model generalizes extremely poorly to new data. We can simulate this by splitting our data into a *training set* and a *testing set*. Scikit-learn contains some convenient routines to do this:

In [ ]:
from sklearn.model_selection import train_test_split
Xtrain, Xtest, ytrain, ytest = train_test_split(X, y)
clf.fit(Xtrain, ytrain)
print(clf.score(Xtest, ytest))

This paints a better picture of the true performance of our classifier: apparently there the classification is not perfect.

This is why it's **extremely important** to use a train/test split when evaluating your models.  We'll go into more depth on model evaluation later in this tutorial.

## Quick Application: Optical Character Recognition

To demonstrate the above principles on a more interesting problem, let's consider OCR (Optical Character Recognition) – that is, recognizing hand-written digits.
In the wild, this problem involves both locating and identifying characters in an image. Here we'll take a shortcut and use scikit-learn's set of pre-formatted digits, which is built-in to the library.

### Loading and visualizing the digits data

We'll use scikit-learn's data access interface and take a look at this data:

In [ ]:
from sklearn import datasets
digits = datasets.load_digits()
digits.images.shape

In [ ]:
from plotly import tools

layout = go.Layout(
    width=500,
    height=500,
)
fig = tools.make_subplots(rows=1, cols=10)

for j in range(1, 11):
    heatmap = go.Heatmap(
        z=digits.images[j],
        showscale=False,
        colorscale = 'Greys',
        reversescale=True)
    fig.append_trace(heatmap, 1, j)
fig['layout'].update(height=250, width=1000, title='digits examples')
iplot(fig, image_height=200, image_width=200)

Let's plot a few of these:

Here the data is simply each pixel value within an 8x8 grid:

In [ ]:
# The images themselves
print(digits.images.shape)
print(digits.images[0])

In [ ]:
# The data for use in our algorithms
print(digits.data.shape)
print(digits.data[0])

In [ ]:
# The target label
print(digits.target)

So our data have 1797 samples in 64 dimensions.

### Classification on Digits

Let's try a classification task on the digits. The first thing we'll want to do is split the digits into a training and testing sample:

In [ ]:
from sklearn.cross_validation import train_test_split
Xtrain, Xtest, ytrain, ytest = train_test_split(
    digits.data, digits.target, random_state=2)
print(Xtrain.shape, Xtest.shape)

Let's use a simple logistic regression which (despite its confusing name) is a classification algorithm:

In [ ]:
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression()
clf.fit(Xtrain, ytrain)

We can check our classification accuracy by comparing the true values of the test set to the predictions:

In [ ]:
clf.score(Xtest, ytest)

This single number doesn't tell us **where** we've gone wrong: one nice way to do this is to use the *confusion matrix*

In [ ]:
from sklearn.metrics import confusion_matrix
ypred = clf.predict(Xtest)
print(confusion_matrix(ytest, ypred))

In [ ]:
heatmap = [go.Heatmap(
        z=confusion_matrix(ytest, ypred),
        showscale=False)]

iplot(heatmap)

The interesting thing is that even with this simple logistic regression algorithm, many of the mislabeled cases are ones that we ourselves might get wrong!

There are many ways to improve this classifier, but we're out of time here. To go further, we could use a more sophisticated model, use cross validation, or apply other techniques.
We'll cover some of these topics later in the tutorial.

## Exercise

We have achieved 95% accuracy using a logistic regression model. Can you do better?